In [ ]:
# 알라딘 100자평

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


def setup_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_experimental_option("detach", True)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--user-agent=Mozilla/5.0")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def extract_star_rating(star_div):
    try:
        star_on_images = star_div.find_elements(By.CSS_SELECTOR, "img[src*='icon_star_on.png']")
        return len(star_on_images) if star_on_images else "없음"
    except:
        return "없음"


def crawl_aladin_100char_reviews_only(driver, isbn_list):
    base_url = "https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=Book&SearchWord={}"
    all_reviews = []

    for idx, isbn in enumerate(isbn_list):
        print(f"\n=== [{isbn}] ({idx + 1}/{len(isbn_list)}) 100자평 수집 시작 ===")
        try:
            driver.get(base_url.format(isbn))
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "front_cover"))).click()
            time.sleep(2)

            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wa_product_top1_wa_Top_Ranking_pnlRanking"]/div[2]/a[3]'))).click()
            time.sleep(1)

            try:
                total_tab = driver.find_element(By.ID, "tabTotal")
                if "on" in total_tab.get_attribute("class"):
                    print(" > 100자평 전체탭 이미 선택됨")
                else:
                    driver.execute_script("arguments[0].scrollIntoView(true);", total_tab)
                    driver.execute_script("arguments[0].click();", total_tab)
                    print(" > 100자평 전체탭 강제 클릭 완료")
                    time.sleep(1)
            except:
                print(" > 100자평 전체탭 클릭 실패")

            print(" > 100자평 더보기 시작...")
            for _ in range(30):
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(1)
                    more_button = driver.find_element(By.CSS_SELECTOR, "a[href='javascript:fn_CommunityReviewMore();']")
                    if more_button.is_displayed():
                        driver.execute_script("arguments[0].click();", more_button)
                        time.sleep(2)
                    else:
                        break
                except:
                    break

            try:
                reviews = driver.find_elements(By.CSS_SELECTOR, "div.hundred_list")
            except:
                reviews = []

            print(f" > 100자평 {len(reviews)}개 수집 완료")
            for r in reviews:
                try:
                    nickname = r.find_element(By.CSS_SELECTOR, "a.Ere_sub_gray8.Ere_fs13.Ere_PR10").text.strip()
                except:
                    nickname = "없음"
                try:
                    star = extract_star_rating(r.find_element(By.CSS_SELECTOR, "div.HL_star"))
                except:
                    star = "없음"
                try:
                    text = r.find_element(By.CSS_SELECTOR, "div.HL_write").text.strip()
                    if nickname != "없음":
                        text = text.replace(nickname, "").strip()
                except:
                    text = "없음"
                all_reviews.append({'isbn': isbn, 'nickname': nickname, 'star_rating': star, 'review_content': text})

            print(f"=== [{isbn}] 100자평 수집 끝 (총 {len(reviews)}개) ===")

        except Exception as e:
            print(f"[{isbn}] 처리 오류: {e}")
            all_reviews.append({'isbn': isbn, 'nickname': '오류', 'star_rating': '오류', 'review_content': str(e)})
        time.sleep(2)

    return all_reviews


def save_to_csv(data, filename="aladin_100char_reviews.csv"):
    if not data:
        print("❌ 저장할 데이터가 없습니다. 리스트가 비어 있습니다.")
        return

    df = pd.DataFrame(data)
    expected_columns = ['isbn', 'nickname', 'star_rating', 'review_content']
    missing_cols = [col for col in expected_columns if col not in df.columns]

    if missing_cols:
        print(f"❌ 저장 실패: 누락된 컬럼 -> {missing_cols}")
        print("🔎 DataFrame 컬럼 목록:", df.columns.tolist())
        print("📦 첫 1개 데이터 샘플:", data[:1])
        return

    df[expected_columns].to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n✅ 저장 완료: {filename} / 총 {len(df)}개")


if __name__ == "__main__":
    df_csv = pd.read_csv("비베스트셀러_랜덤.csv", encoding="utf-8-sig")
    isbn_list = df_csv['ISBN'].dropna().unique().tolist()

    print("🔍 100자평 크롤링 시작...")
    driver = setup_driver()
    try:
        results = crawl_aladin_100char_reviews_only(driver, isbn_list)
        save_to_csv(results)
    finally:
        driver.quit()

In [ ]:
# 알라딘 마이리뷰 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re


def setup_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_experimental_option("detach", True)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--user-agent=Mozilla/5.0")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def extract_star_rating(star_div):
    try:
        star_on_images = star_div.find_elements(By.CSS_SELECTOR, "img[src*='icon_star_on.png']")
        return len(star_on_images) if star_on_images else "없음"
    except:
        return "없음"


def crawl_aladin_Myreviews_only(driver, isbn_list):
    base_url = "https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=Book&SearchWord={}"
    all_reviews = []

    for idx, isbn in enumerate(isbn_list):
        print(f"\n=== [{isbn}] ({idx + 1}/{len(isbn_list)}) 마이리뷰 수집 시작 ===")
        try:
            driver.get(base_url.format(isbn))
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "front_cover"))).click()
            time.sleep(2)

            print(" > 마이리뷰 전체탭 탐색...")
            found_section = False
            for _ in range(40):
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(1.5)
                try:
                    tab = driver.find_element(By.ID, "tabMyReviewTotal")
                    if tab.is_displayed():
                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
                        time.sleep(2)
                        tab.click()
                        print(" > 마이리뷰 전체탭 클릭 성공")
                        found_section = True
                        break
                except:
                    continue
            if not found_section:
                print(" > 마이리뷰 전체탭 탐색 실패")

            print(" > 리뷰 수집 시작...")
            for _ in range(30):
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
                    list_more_button = driver.find_element(By.CSS_SELECTOR, "a[href='javascript:MyReviewMore();']")
                    if list_more_button.is_displayed():
                        driver.execute_script("arguments[0].click();", list_more_button)
                        print(" > 리뷰 목록 더보기 클릭 성공")
                        time.sleep(3)
                    else:
                        print(" > 리뷰 목록 더보기 없음")
                        break
                except:
                    print(" > 리뷰 목록 더보기 없음")
                    break

            # 수정된 부분: 자세히 보기 버튼 클릭 로직 개선
            print(" > 자세히 보기 버튼들 클릭 시작...")
            try:
                # 모든 자세히 보기 버튼 찾기
                detail_buttons = driver.find_elements(By.CSS_SELECTOR, "a.Ere_sub_gray8.Ere_fs13")
                print(f" > 발견된 자세히 보기 버튼 수: {len(detail_buttons)}")
                
                clicked_count = 0
                for i, btn in enumerate(detail_buttons):
                    try:
                        # onclick 속성 확인
                        onclick_attr = btn.get_attribute("onclick")
                        if onclick_attr and "fn_show_mypaper_utf8" in onclick_attr:
                            print(f" > 버튼 {i+1} 클릭 시도: {onclick_attr[:50]}...")
                            
                            # 버튼이 보이도록 스크롤
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                            time.sleep(1)
                            
                            # 버튼 클릭 시도 (여러 방법으로)
                            try:
                                btn.click()
                                print(f" > 버튼 {i+1} 일반 클릭 성공")
                            except:
                                try:
                                    driver.execute_script("arguments[0].click();", btn)
                                    print(f" > 버튼 {i+1} JS 클릭 성공")
                                except:
                                    # onclick 함수 직접 실행
                                    driver.execute_script(onclick_attr)
                                    print(f" > 버튼 {i+1} onclick 직접 실행 성공")
                            
                            clicked_count += 1
                            time.sleep(1.5)  # 각 클릭 후 대기
                            
                    except Exception as btn_err:
                        print(f" > 버튼 {i+1} 클릭 실패: {btn_err}")
                        continue
                
                print(f" > 총 {clicked_count}개 자세히 보기 버튼 클릭 완료")
                time.sleep(3)  # 모든 버튼 클릭 후 추가 대기
                
            except Exception as e:
                print(f" > 자세히 보기 버튼 처리 실패: {e}")

            # 리뷰 수집
            try:
                reviews = driver.find_elements(By.CSS_SELECTOR, "div.hundred_list")
            except:
                reviews = []

            print(f" > 마이리뷰 {len(reviews)}개 수집 완료")
            for r in reviews:
                try:
                    nickname = r.find_element(By.CSS_SELECTOR, "a.Ere_sub_gray8.Ere_fs13.Ere_PR10").text.strip()
                except:
                    nickname = "없음"
                try:
                    star = extract_star_rating(r.find_element(By.CSS_SELECTOR, "div.HL_star"))
                except:
                    star = "없음"
                
                # 수정된 부분: 리뷰 텍스트 추출 로직 개선
                try:
                    text = "없음"
                    
                    # 1. 확장된 paper-contents 영역에서 텍스트 찾기 (우선순위 1)
                    try:
                        paper_contents = r.find_elements(By.CSS_SELECTOR, "div.paper-contents")
                        for pc in paper_contents:
                            if pc.is_displayed() and pc.text.strip():
                                text = pc.text.strip()
                                print(f" > paper-contents에서 텍스트 추출: {text[:50]}...")
                                break
                    except:
                        pass
                    
                    # 2. paperAll_* ID를 가진 div에서 찾기 (우선순위 2)
                    if text == "없음":
                        try:
                            paper_all_divs = r.find_elements(By.CSS_SELECTOR, "div[id^='paperAll_']")
                            for pad in paper_all_divs:
                                if pad.is_displayed() and pad.text.strip():
                                    text = pad.text.strip()
                                    print(f" > paperAll div에서 텍스트 추출: {text[:50]}...")
                                    break
                        except:
                            pass
                    
                    # 3. 기존 방식들 (우선순위 3)
                    if text == "없음":
                        try:
                            paragraphs = r.find_elements(By.CSS_SELECTOR, "div.txc-textbox p")
                            if paragraphs:
                                text = " ".join([p.text.strip() for p in paragraphs if p.text.strip()])
                                print(f" > txc-textbox p에서 텍스트 추출: {text[:50]}...")
                        except:
                            pass
                    
                    # 4. divPaper로 시작하는 ID 찾기 (우선순위 4)
                    if text == "없음":
                        try:
                            div_paper = r.find_element(By.CSS_SELECTOR, "div[id^='divPaper']")
                            if div_paper.text.strip():
                                text = div_paper.text.strip()
                                print(f" > divPaper에서 텍스트 추출: {text[:50]}...")
                        except:
                            pass
                    
                    # 닉네임 제거
                    if nickname != "없음" and text != "없음":
                        text = text.replace(nickname, "").strip()
                        
                except Exception as text_err:
                    print(f" > 텍스트 추출 실패: {text_err}")
                    text = "없음"

                all_reviews.append({
                    'isbn': isbn, 
                    'nickname': nickname, 
                    'star_rating': star, 
                    'review_content': text
                })

            print(f"=== [{isbn}] 마이리뷰 수집 끝 (총 {len(reviews)}개) ===")

        except Exception as e:
            print(f"[{isbn}] 처리 오류: {e}")
            all_reviews.append({
                'isbn': isbn, 
                'nickname': '오류', 
                'star_rating': '오류', 
                'review_content': str(e)
            })
        time.sleep(2)

    return all_reviews


def save_to_csv(data, filename="aladin_myreviews.csv"):      # <- 베스트셀러 파일 바꾼다음에 이거 이름 바꿔주기!!!!!!!!!!!!!
    if not data:
        print("❌ 저장할 데이터가 없습니다. 리스트가 비어 있습니다.")
        return

    df = pd.DataFrame(data)
    expected_columns = ['isbn', 'nickname', 'star_rating', 'review_content']
    missing_cols = [col for col in expected_columns if col not in df.columns]

    if missing_cols:
        print(f"❌ 저장 실패: 누락된 컬럼 -> {missing_cols}")
        print("🔎 DataFrame 컬럼 목록:", df.columns.tolist())
        print("📦 첫 1개 데이터 샘플:", data[:1])
        return

    df[expected_columns].to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n✅ 저장 완료: {filename} / 총 {len(df)}개")


if __name__ == "__main__":
    df_csv = pd.read_csv("비베스트셀러_랜덤.csv", encoding="utf-8-sig")  # <- 여기에 베스트셀러 파일로 바꾸기!!!!!!!!!!!
    isbn_list = df_csv['ISBN'].dropna().unique().tolist()
    
    print("🔍 마이리뷰 크롤링 시작...")
    driver = setup_driver()
    try:
        results = crawl_aladin_Myreviews_only(driver, isbn_list)
        save_to_csv(results)
    finally:
        driver.quit()

🔍 마이리뷰 크롤링 시작...

=== [9788901294742] (1/2) 마이리뷰 수집 시작 ===
 > 마이리뷰 전체탭 탐색...
 > 마이리뷰 전체탭 클릭 성공
 > 리뷰 수집 시작...
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 클릭 성공
 > 리뷰 목록 더보기 없음
 > 자세히 보기 버튼들 클릭 시작...
 > 발견된 자세히 보기 버튼 수: 441
 > 버튼 34 클릭 시도: fn_show_mypaper_utf8(8307820,'MyReview')...
 > 버튼 34 일반 클릭 성공
 > 버튼 39 클릭 시도: fn_show_mypaper_utf8(16440050,'MyReview')...
 > 버튼 39 일반 클릭 성공
 > 버튼 44 클릭 시도: fn_show_mypaper_utf8(8605600,'MyReview')...
 > 버튼 44 일반 클릭 성공
 > 버튼 47 클릭 시도: fn_show_mypaper_utf8(16441990,'MyReview')...
 > 버튼 47 일반 클릭 성공


isbn_list = ['9788901294742','9791168270749']